### This notebook demonstrates the machine learning pipeline and includes the Airflow DAG definition.
### Airflow is not executed here due to OS limitations (Airflow requires Linux/WSL2), but the pipeline structure, scheduling logic, and tasks are clearly defined.

In [1]:
pip install apache-airflow


INFO: pip is looking at multiple versions of cadwyn to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/5.4 MB ? eta -:--:--
   ------- -------------------------------- 1.0/5.4 MB 6.0 MB/s eta 0:00:01
   --------------- ------------------------ 2.1/5.4 MB 4.6 MB/s eta 0:00:01
   ----------------------- ---------------- 3.1/5.4 MB 4.6 MB/s eta 0:00:01
   ---------------------------- ----------- 3.9/5.4 MB 4.6 MB/s eta 0:00:01
   ------------------------------------ --- 5.0/5.4 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 5.4/5.4 MB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   ------------ --------------------------- 1.0/3.5 MB 5.0 MB/s eta 0:00:01
   --------------------------- ------------ 2.4/3.5 MB 5.5 MB/s eta 0:00:01
   ------------------------------------ --- 3.1/3.5 MB 5.2 MB/s eta 0:00:01
   --------------------------------


[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: C:\Users\anjal\AppData\Local\Programs\Python\Python313\python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# load the dataset
data = pd.read_csv('screentime_analysis.csv')

# check for missing values and duplicates
print(data.isnull().sum())
print(data.duplicated().sum())

# convert Date column to datetime and extract features
data['Date'] = pd.to_datetime(data['Date'])
data['DayOfWeek'] = data['Date'].dt.dayofweek
data['Month'] = data['Date'].dt.month

# encode the categorical 'App' column using one-hot encoding
data = pd.get_dummies(data, columns=['App'], drop_first=True)

# scale numerical features using MinMaxScaler
scaler = MinMaxScaler()
data[['Notifications', 'Times Opened']] = scaler.fit_transform(data[['Notifications', 'Times Opened']])

# feature engineering
data['Previous_Day_Usage'] = data['Usage (minutes)'].shift(1)
data['Notifications_x_TimesOpened'] = data['Notifications'] * data['Times Opened']

# save the preprocessed data to a file
data.to_csv('preprocessed_screentime_analysis.csv', index=False)

Date               0
App                0
Usage (minutes)    0
Notifications      0
Times Opened       0
dtype: int64
0


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# split data into features and target variable
X = data.drop(columns=['Usage (minutes)', 'Date'])
y = data['Usage (minutes)']

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train the model
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# evaluate the model
predictions = model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 15.398500000000002


In [5]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def preprocess_data():
    data = pd.read_csv('screentime_analysis.csv')

    data['Date'] = pd.to_datetime(data['Date'])
    data['DayOfWeek'] = data['Date'].dt.dayofweek
    data['Month'] = data['Date'].dt.month

    data = data.drop(columns=['Date'])
    data = pd.get_dummies(data, columns=['App'], drop_first=True)

    scaler = MinMaxScaler()
    data[['Notifications', 'Times Opened']] = scaler.fit_transform(
        data[['Notifications', 'Times Opened']]
    )

    data.to_csv('preprocessed_screentime_analysis.csv', index=False)

dag = DAG(
    dag_id='data_preprocessing',
    schedule='@daily',
    start_date=datetime(2025, 1, 1),
    catchup=False
)

preprocess_task = PythonOperator(
    task_id='preprocess',
    python_callable=preprocess_data,
    dag=dag
)


C:\Users\anjal\AppData\Local\Temp\ipykernel_18664\1142362576.py:2 DeprecatedImportWarning: The `airflow.operators.python.PythonOperator` attribute is deprecated. Please use `'airflow.providers.standard.operators.python.PythonOperator'`.